# Programming Assignment: Text analysis

Continuing working with support vector machine. This task is a bit more challenging.  
Original five tasks will be presented in Russian.

**Tasks will be duplicated in english further in this assignment**

1. Загрузите объекты из новостного датасета 20 newsgroups, относящиеся к категориям "космос" и "атеизм" (инструкция приведена выше). Обратите внимание, что загрузка данных может занять несколько минут
2. Вычислите TF-IDF-признаки для всех текстов. Обратите внимание, что в этом задании мы предлагаем вам вычислить TF-IDF по всем данным. При таком подходе получается, что признаки на обучающем множестве используют информацию из тестовой выборки — но такая ситуация вполне законна, поскольку мы не используем значения целевой переменной из теста. На практике нередко встречаются ситуации, когда признаки объектов тестовой выборки известны на момент обучения, и поэтому можно ими пользоваться при обучении алгоритма.
3. Подберите минимальный лучший параметр C из множества [10^-5, 10^-4, ... 10^4, 10^5] для SVM с линейным ядром (kernel='linear') при помощи кросс-валидации по 5 блокам. Укажите параметр random_state=241 и для SVM, и для KFold. В качестве меры качества используйте долю верных ответов (accuracy).
4. Обучите SVM по всей выборке с оптимальным параметром C, найденным на предыдущем шаге.
5. Найдите 10 слов с наибольшим абсолютным значением веса (веса хранятся в поле coef_ у svm.SVC). Они являются ответом на это задание. Укажите эти слова через запятую или пробел, в нижнем регистре, в лексикографическом порядке.


In [1]:
import numpy as np

from sklearn import datasets
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

import pandas

## Task №1
*Ru*: Загрузите объекты из новостного датасета 20 newsgroups, относящиеся к категориям "космос" и "атеизм" (инструкция приведена выше). Обратите внимание, что загрузка данных может занять несколько минут

*En:* Download objects from the 20 newsgroups news dataset categorized as "space" and "atheism" 


In [2]:
newsgroups = datasets.fetch_20newsgroups(
                    subset='all', 
                    categories=['alt.atheism', 'sci.space']
             )

In [3]:
#texts are stored there
X = newsgroups.data
#X

In [4]:
#class numbers
y = newsgroups.target
y

array([0, 0, 1, ..., 1, 1, 0], dtype=int64)

## Task №2
*Ru*:  Вычислите TF-IDF-признаки для всех текстов. Обратите внимание, что в этом задании мы предлагаем вам вычислить TF-IDF по всем данным. При таком подходе получается, что признаки на обучающем множестве используют информацию из тестовой выборки — но такая ситуация вполне законна, поскольку мы не используем значения целевой переменной из теста. На практике нередко встречаются ситуации, когда признаки объектов тестовой выборки известны на момент обучения, и поэтому можно ими пользоваться при обучении алгоритма.

*En:* Calculate TF-IDF tags for all texts. 

In [5]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

## Task №3
*Ru*: Подберите минимальный лучший параметр C из множества [10^-5, 10^-4, ... 10^4, 10^5] для SVM с линейным ядром (kernel='linear') при помощи кросс-валидации по 5 блокам. Укажите параметр random_state=241 и для SVM, и для KFold. В качестве меры качества используйте долю верных ответов (accuracy).

*En:*

In [6]:
cv = KFold(n_splits=5, shuffle=True, random_state = 241)

In [7]:
model = svm.SVC(kernel='linear', random_state=241)

In [8]:
grid = {'C': np.power(10.0, np.arange(-5, 6))}
gs = GridSearchCV(model, grid, scoring='accuracy', cv=cv)
gs.fit(X, y)

GridSearchCV(cv=KFold(n_splits=5, random_state=241, shuffle=True),
             estimator=SVC(kernel='linear', random_state=241),
             param_grid={'C': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05])},
             scoring='accuracy')

In [9]:
C = gs.best_params_['C']

## Task №4
*Ru*: Обучите SVM по всей выборке с оптимальным параметром C, найденным на предыдущем шаге.

*En:* Train the SVM over the entire sample with the optimal C parameter found in the previous step. 

In [10]:
model = svm.SVC(kernel='linear', random_state=241, C=C)
model.fit(X, y)

SVC(kernel='linear', random_state=241)

## Task №5
*Ru*: Найдите 10 слов с наибольшим абсолютным значением веса (веса хранятся в поле coef_ у svm.SVC). Они являются ответом на это задание. Укажите эти слова через запятую или пробел, в нижнем регистре, в лексикографическом порядке.

*En:* Find the 10 words with the highest absolute weight (weights are stored in the coef_ field of svm.SVC). They are the answer to this task. Specify these words, separated by commas or spaces, in lower case, in lexicographic order. 

In [11]:
#for i in model.coef_:
#    print(i)

In [12]:
words = vectorizer.get_feature_names()
coef = pandas.DataFrame(model.coef_.data, model.coef_.indices)
top_words = coef[0].map(lambda w: abs(w)).sort_values(ascending=False).head(10).index.map(lambda i: words[i])

In [13]:
np.sort(top_words)

array(['atheism', 'atheists', 'bible', 'god', 'keith', 'moon', 'religion',
       'sci', 'sky', 'space'], dtype=object)